In [1]:
import urllib.request, json , time, os, difflib, itertools
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime
#difflib.get_close_matches

In [2]:
try:
    import httplib
except:
    import http.client as httplib

def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        # print("True")
        return True
    except:
        conn.close()
        # print("False")
        return False

In [3]:
check_internet()

True

In [4]:
def get_historic_price(query_url,json_path,csv_path):
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)
    
    stock_id=query_url.split("&period")[0].split("symbol=")[1]
    
    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Historical data of "+stock_id+" already exists, Updating data...")

    try:
        with urllib.request.urlopen(query_url) as url:
            parsed = json.loads(url.read().decode())
    except:
        print("|||  Historical data of "+stock_id+" doesn't exist")
        return
    
    else:
        if os.path.exists(json_path+stock_id+'.json'):
            os.remove(json_path+stock_id+'.json')
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)

        try:
            Date=[]
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%d-%m-%Y'))

            Low=parsed['chart']['result'][0]['indicators']['quote'][0]['low']
            Open=parsed['chart']['result'][0]['indicators']['quote'][0]['open']
            Volume=parsed['chart']['result'][0]['indicators']['quote'][0]['volume']
            High=parsed['chart']['result'][0]['indicators']['quote'][0]['high']
            Close=parsed['chart']['result'][0]['indicators']['quote'][0]['close']
            Adjusted_Close=parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

            df=pd.DataFrame(list(zip(Date,Low,Open,Volume,High,Close,Adjusted_Close)),columns =['Date','Low','Open','Volume','High','Close','Adjusted Close'])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
            return
        except:
            print(">>>  Historical data of "+stock_id+" exists but has no trading data")

In [5]:
json_path = os.getcwd()+os.sep+".."+os.sep+"historic_data"+os.sep+"json"+os.sep
csv_path = os.getcwd()+os.sep+".."+os.sep+"historic_data"+os.sep+"csv"+os.sep


In [6]:
if not os.path.isdir(json_path):
    os.makedirs(json_path)
if not os.path.isdir(csv_path):
    os.makedirs(csv_path)


In [7]:
period1 = 0
period2 = 9999999999
interval = "1d"

In [8]:
country_name = "germany"


In [1]:
full_ticker_dict={
"Adidas":'ADS.DE',
 "Airbus":'AIR.PA',
 "Allianz":'ALV.DE',
 "BASF":'BAS.DE',
"Bayer":'BAYN.DE',
 "BMW":'BMW.DE',
 "Brenntag":'BNR.DE',
"Continental":'CON.DE',
    "Covestro": '1COV.DE',
    "Daimler Truck":'DTG.DE',
    "Delivery Hero": 'DHER.DE',
     "Deutsche Börse":'DB1.DE',
    "Deutsche Bank":"DBK.DE",
 "Deutsche Post": 'DHL.DE',
 "Deutsche Telekom":'DTE.DE',
 "E.ON":'EOAN.DE',
     "Fresenius": "FRE.DE",
    "Fresenius Medical Care":'FME.DE',
 "Hannover Rück":'HNR1.DE',
 "HeidelbergCement": 'HEI.DE',
    "Hello Fresh": 'HFG.DE',
     "Henkel":'HEN3.DE',
    "Infineon": 'IFX.DE',
"Linde":'LIN.DE',
"Mercedes Benz": "MBG.DE",
    "Merck": 'MRK.DE',
 "MTU Aero Engines":'MTX.DE',
 "Münchner Rück":'MUV2.DE',
     "Porsche":'P911.DE',
     "Puma":'PUM.DE',
     "QIAGEN":'QIA.DE',
    "RWE": 'RWE.DE',
     "SAP":'SAP.DE',
 "Sartorius":'SRT3.DE',
    "Siemens": 'SIE.DE',
    "Siemens Healthineers": 'SHL.DE',
    "Symrise":'SY1.DE',
    "Volkswagen":'VOW3.DE',
    "Vonovia": 'VNA.DE',
 "Zalando":'ZAL.DE',}

In [2]:
full_ticker_dict.values()

dict_values(['ADS.DE', 'AIR.PA', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BMW.DE', 'BNR.DE', 'CON.DE', '1COV.DE', 'DTG.DE', 'DHER.DE', 'DB1.DE', 'DBK.DE', 'DHL.DE', 'DTE.DE', 'EOAN.DE', 'FRE.DE', 'FME.DE', 'HNR1.DE', 'HEI.DE', 'HFG.DE', 'HEN3.DE', 'IFX.DE', 'LIN.DE', 'MBG.DE', 'MRK.DE', 'MTX.DE', 'MUV2.DE', 'P911.DE', 'PUM.DE', 'QIA.DE', 'RWE.DE', 'SAP.DE', 'SRT3.DE', 'SIE.DE', 'SHL.DE', 'SY1.DE', 'VOW3.DE', 'VNA.DE', 'ZAL.DE'])

In [24]:
query_urls=[]
for ticker in full_ticker_dict.values():
    query_urls.append("https://query1.finance.yahoo.com/v8/finance/chart/"+ticker+"?symbol="+ticker+"&period1=0&period2=9999999999&interval=1d&includePrePost=true&events=div%2Csplit")

In [25]:
with Pool(processes=len(query_urls)) as pool:
    pool.starmap(get_historic_price, zip(query_urls, itertools.repeat(json_path), itertools.repeat(csv_path)))
print("All downloads completed !")

>>>  Historical data of P911.DE saved
>>>  Historical data of HNRB.DE saved
>>>  Historical data of DTG.DE saved
>>>  Historical data of SRP.DE saved
>>>  Historical data of HFG.DE saved
>>>  Historical data of SHL.DE saved
>>>  Historical data of DHER.DE saved
>>>  Historical data of ZAL.DE saved
>>>  Historical data of 1COV.DE saved
>>>  Historical data of BNR.DE saved
>>>  Historical data of VNA.DE saved
>>>  Historical data of MTX.DE saved
>>>  Historical data of FMEA.SG saved
>>>  Historical data of SY1.DE saved
>>>  Historical data of LIN.DE saved
>>>  Historical data of MUV2.DE saved
>>>  Historical data of DHL.DE saved>>>  Historical data of DB1.DE saved

>>>  Historical data of HEN3.DE saved
>>>  Historical data of EOAN.DE saved
>>>  Historical data of RWE.DE saved
>>>  Historical data of MRK.DE saved
>>>  Historical data of ALV.DE saved
>>>  Historical data of IFX.DE saved
>>>  Historical data of DTE.DE saved
>>>  Historical data of BMW.DE saved>>>  Historical data of AIR.PA 

In [26]:
df_translate=pd.DataFrame(full_ticker_dict,index=[0]).T.reset_index()
df_translate.columns=["name","shortener"]
df_translate

,name,shortener
0,Adidas,ADS.DE
1,Airbus,AIR.PA
2,Allianz,ALV.DE
3,BASF,BAS.DE
4,Bayer,BAYN.DE
5,BMW,BMW.DE
6,Brenntag,BNR.DE
7,Continental,CON.DE
8,Covestro,1COV.DE
9,Daimler Truck,DTG.DE


In [78]:
df_translate.to_csv("data/translate.csv",index=False)